# Assignment 4 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `mammography.mat` that you can download from [http://odds.cs.stonybrook.edu/mammography-dataset/](http://odds.cs.stonybrook.edu/mammography-dataset/). Below you can find the code to load this file into `X` and `Y`, you just need to put the file in your `data` folder. During mammography the doctor would be looking for something called `calcification`, which is calcium deposits in the breast tissue and is used as an indicator of cancer. In this dataset the `X` corresponds to some measurements, there are 6 features. The `Y` is a 0-1 label where 1 represents calcification and 0 does not.

1. [3p] Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, I have prepared a shuffling with a fixed seed, this way I can make sure that we all did the same splits. That is [train,test,validation] is the splitting layout.
2. [4p] Train a machine learning model on the training data (you are free to choose it yourself). Hint: you could always try `LogisticRegression`, but for it to work well you would need `StandardScaler` and put everything in a `Pipeline`.
3. [3p] Use the classification report from `Utils` and compute the intervals for precision-recall etc on the test set with `union_bound correction` with 95% confidence.
4. [3p] You are interested in minimizing the average cost of your classifier, but first we must define it:
    * If someone is calcified but classified as not, we say it costs 30 **(this is the worst scenario)** 
    * If someone is not calcified but classified as calcified, we say it costs 5 **(this probably only costs extra investigation)**
    * If someone is calcified but classified as calcified, costs 0 **(We did the right thing, no cost)**
    * If someone is not calcified but classified as not, costs 0 **(We did the right thing, no cost)**.

complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). What would be the cost of having a model that always classifies someone as not calcified on the test set?

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, we do that by checking say 100 evenly spaced proposal thresholds between 0 and 1, and use our testing data to compute the cost.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 95% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted, if we define the random variable
$$
    C = 30(1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
$$
then $C$ denotes the cost of a randomly chosen patient. In the above we are estimating $\mathbb{E}[C]$ using the empirical mean. However, since the number of calcifications in the population is fairly small and our classifier probably has a fairly high precision for the $0$ class, then $C$ should have fairly small variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [17]:
import scipy.io as so
import numpy as np
data = so.loadmat('data/mammography.mat')

np.random.seed(0)
shuffle_index = np.arange(0,len(data['X']))
np.random.shuffle(shuffle_index)

X = data['X'][shuffle_index,:]
Y = data['y'][shuffle_index,:].flatten()

In [18]:
from Utils import train_test_validation
# Part 1

# Split the X,Y into three parts, make sure that the sizes are
# (6709, 6), (1677, 6), (2797, 6), (6709,), (1677,), (2797,)
#X_train_test, X_valid, Y_train_test, Y_valid = train_test_split(X, Y, test_size=0.2501117768040776)
#X_train, X_test, Y_train, Y_test = train_test_split(X_train_test, Y_train_test, test_size=1677/len(X_train_test))

X_train, X_test, X_valid, Y_train, Y_test, Y_valid = train_test_validation(X, Y, test_size=0.15, validation_size=0.25, shuffle=False)

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# Part 2
# Use X_train,Y_train to train a model from sklearn. Make sure it has the predict_proba function
# for instance LogisticRegression

# Regarding the StandardScaler in pipeline, it'll automatically [fit & transform](standardize)  the training data
model = Pipeline([('scalar', StandardScaler()), ('LR', LogisticRegression())])
#model = Pipeline([('LR', LogisticRegression())])
model.fit(X_train, Y_train)


# Below will generate the same fit model as Pipleline
# StandardScaler
#sc = StandardScaler()
# Calculate the mean and standard deviation of each features(columns)
#sc.fit(X_train)

# Define Logistic Regression model and fit the model with transformed data
#model1 = LogisticRegression()
#model1.fit(sc.transform(X_train), Y_train)



Pipeline(steps=[('scalar', StandardScaler()), ('LR', LogisticRegression())])

In [20]:
#(-1)
# Part 3

# Compute precision and recall on the test set using 
from Utils import classification_report_interval

cri = classification_report_interval(Y_test, model.predict(X_test), alpha = 0.05)
# Each precision and recall should be a tuple, for instance you can write
# precision0 = (0.9,0.95)


precision0 = (0.94,1.00)
recall0 = (0.96,1.00) #lower not correct
precision1 = (0.33,1.00) #lower not correct
recall1 = (0.09,0.62) #upper not correct

# The code below will check that you supply the proper type
assert(type(precision0) == tuple)
assert(len(precision0) == 2)
assert(type(recall0) == tuple)
assert(len(recall0) == 2)
assert(type(precision1) == tuple)
assert(len(precision1) == 2)
assert(type(recall1) == tuple)
assert(len(recall1) == 2)

#print(cri)

In [31]:

# Part 4

def cost(model,threshold,X,Y):
    #import numpy as np
    pred_proba = model.predict_proba(X)[:,1]
    predictions = (pred_proba >= threshold)*1
    
    # Fill in what is missing to compute the cost and return it
    

    # predictions == 0 but Y == 1 cost 30
    # predictions == 1 but Y == 0 cost 5
    # Rather having a wrong predicted calcified than not unpredicited calcified (Recall high is our goal)
    cost_re = 0
    for i in range(len(predictions)):
        # predictions == 1 but Y == 0 cost 5
        if Y[i] < predictions[i]:
            cost_re += 5
        # predictions == 0 but Y == 1 cost 30   
        elif Y[i] > predictions[i]:
            cost_re += 30
    '''
    cost_30 = int(len(predictions[Y==1]) - np.sum(predictions[Y==1])) # Count how many zeros r there
    cost_5 = int(np.sum(predictions[Y==0]))
    '''
    # Note that we are interested in average cost (cost per person) 
    # => total cost / total numbers of person

    cost_re = cost_re / len(predictions)
    return cost_re
    #return cost_30, cost_5

In [32]:

# Part 4

# Fill in the naive cost of the model that would classify all as non-calcified on the test set
#naive_cost = np.sum(Y_test)*30 / len(Y_test) #-1
naive_cost = cost(model, 2, X_test, Y_test)
#naive_cost

In [51]:
#(-2)
# Part 5
# create array with 100 evenly interval between 0 and 1
# Compute cost(model,each_threshold,X_test,Y_test):
# Find the minimum cost 
threshold = np.linspace(0, 1, num = 101)

optimal_threshold = -1 #0.11
cost_at_optimal_threshold = 30 #0.3518187239117472

for thres in threshold:
    thres_cost = cost(model, thres, X_test, Y_test)
    if thres_cost < cost_at_optimal_threshold:
        cost_at_optimal_threshold = thres_cost
        optimal_threshold = thres


In [25]:
from Utils import epsilon_bounded
# Part 6

cost_at_optimal_threshold_validation = cost(model, optimal_threshold, X_valid, Y_valid) #0.2913836253128352
# Report the cost interval as a tuple cost_interval = (a,b)
ep = epsilon_bounded(len(X_valid), 30, 0.05) # bound will be [0, 30]. Upperbound is 30 when u predict all 0 when all true values are 1. The average cost will be 30*n/n = 30
cost_interval = (0 if cost_at_optimal_threshold_validation - ep < 0 else cost_at_optimal_threshold_validation - ep, cost_at_optimal_threshold_validation + ep)

# The code below will tell you if you filled in the intervals correctly
assert(type(cost_interval) == tuple)
assert(len(cost_interval) == 2)

In [26]:
# Using validation data
# Part 7
pred_proba = model.predict_proba(X_valid)[:,1]
predictions = (pred_proba >= optimal_threshold)*1

cost_of_each = []
for i in range(len(predictions)):
    costt = 30 * (1 - predictions[i]) * Y_valid[i] + 5 * (1 - Y_valid[i]) * predictions[i]
    cost_of_each.append(costt)
cost_of_each = np.array(cost_of_each)
variance_of_C = np.var(cost_of_each)

In [40]:
import scipy.optimize as so

def bennett_epsilon(n, b, sigma, alpha):
    h = lambda u: (1+u)*np.log(1+u)-u
    f = lambda epsilon: np.exp(-n*sigma**2/b**2*h(b*epsilon/sigma**2))-alpha/2
    ans = so.fsolve(f,0.002)
    epsilon = np.abs(ans[0])
    return epsilon

# Part 7

# A 95% confidence interval of the random variable C using Bennett's inequality
b = np.max(cost_of_each) - np.min(cost_of_each)
sd = np.sqrt(variance_of_C)
b_ep = bennett_epsilon(n = len(X_valid), b = b, sigma = sd, alpha = 0.05)
interval_of_C = (variance_of_C - b_ep, variance_of_C + b_ep)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)